In [1]:
import pandas as pd

In [72]:
df_ship = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ship')
df_ship_distance = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ship_distance')
df_asj = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ASJ')
df_ren = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='REN')
df_aka = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='AKA')
df_fwcost = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='fw_cost')


In [9]:
df_ship

,ship,tank_capacity
0,ASJ,93000
1,REN,85000
2,AKA,70000


In [8]:
ships = df_ship['ship'].to_list()
ships

['ASJ', 'REN', 'AKA']

In [14]:
df_ship.loc[df_ship['ship'] == 'ASJ'].values[0]

array(['ASJ', 93000], dtype=object)

In [49]:
ports = df_asj.loc[df_asj["VOYAGE"] == "1-23", "PORTS"][0]
# ports = ports.split(";")
ports

'SURABAYA;BALIKPAPAN;PALU'

In [64]:
def get_list(df, reference, reference_column, destination_column):
    lst = df.loc[df[reference_column] == reference, destination_column].values
    return lst

In [165]:
ports = get_list(df_asj, "1-23", "VOYAGE", "PORTS")
ports = ports[0].split(";")
ports

['SURABAYA', 'BALIKPAPAN', 'PALU']

In [170]:
cost = [get_list(df_fwcost, item, "LOKASI", "COST")[0] for item in ports]
cost

[40000, 24000, 25000]

: 

In [105]:
trips = get_list(df_asj, "1-23", "VOYAGE", "TRIPS")
trips = trips[0].split(";")
trips


['SURABAYA-BALIKPAPAN', 'BALIKPAPAN-PALU', 'PALU-SURABAYA']

In [73]:
df_ship_distance

,TRIP,DISTANCE
0,JAKARTA-SURABAYA,438.000000
1,SURABAYA-BALIKPAPAN,600.000000
2,BALIKPAPAN-PALU,153.000000
3,PALU-SURABAYA,780.160000
4,SURABAYA-MAKASSAR,641.153333
...,...,...
64,BATAM-JAKARTA,591.000000
65,SURABAYA-SEMARANG,284.000000
66,SEMARANG-MAKASSAR,671.000000
67,MAKASSAR-JAKARTA,806.000000


In [98]:

distance = []
for trip in trips:
    item = get_list(df_ship_distance, trip, "TRIP", "DISTANCE")
    wrapper = list(item)
    distance.append(wrapper)

distances = np.array(distance).reshape(-1, 1)
distance


[[600.0], [153.0], [780.1599999997532]]

In [140]:
def get_distances(trips):
    distance = []
    for trip in trips:
        item = get_list(df_ship_distance, trip, "TRIP", "DISTANCE")
        wrapper = list(item)
        distance.append(wrapper)
    return distance
        

In [141]:
distance = get_distances(trips)
distance

[[600.0], [153.0], [780.1599999997532]]

In [158]:
def predict_freshwater(distances):
    model = joblib.load("../models/model.pkl")
    scaler = joblib.load("../models/scaler.pkl")
    distance_scaled = scaler.transform(distances)
    fwneeds = []
    for item in distance_scaled:
        fw = model.predict(item.reshape(1, -1))
        fwneeds.append(round(fw[0], 2))
    return fwneeds
    

In [159]:
distance_numbers = [item[0] for item in distance]

In [162]:
df_fw = pd.DataFrame({
    "Trip": trips,
    "Distance": distance_numbers,
    "Freshwater": fwneed,
})
df_fw

,Trip,Distance,Freshwater
0,SURABAYA-BALIKPAPAN,600.00,17427.57
1,BALIKPAPAN-PALU,153.00,5511.14
2,PALU-SURABAYA,780.16,22230.39


In [161]:
fwneed = predict_freshwater(distance)
fwneed

c:\Users\Afifudin\anaconda3\envs\spil-HRIS\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[17427.57, 5511.14, 22230.39]

In [99]:
import joblib

model = joblib.load("../models/model.pkl")
scaler = joblib.load("../models/scaler.pkl")

In [100]:
distance_scaled = scaler.transform(distance)

c:\Users\Afifudin\anaconda3\envs\spil-HRIS\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [116]:
fw_need = []
for item in distance_scaled:
    fw = model.predict(item.reshape(1, -1))
    fw_need.append(fw[0])

fw_need

[17427.56807881554, 5511.143006085158, 22230.39389783157]

In [2]:
import locale
locale.setlocale(locale.LC_ALL, '')
currency_string = locale.currency(1000000, grouping=True)
currency_string

'Rp1.000.000,00'